In [69]:
import pandas as pd
import psycopg2

In [70]:
def create_database():
#connect to the default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=password")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    cur.execute("DROP DATABASE IF EXISTS abc")
    cur.execute("CREATE DATABASE abc")

    conn.close() #to close the connection with default database

    conn = psycopg2.connect("host=127.0.0.1 dbname=abc user=postgres password=password")
    cur = conn.cursor()

    return cur,conn

In [84]:
items = pd.read_csv("C:/Users/GUNGUN PANDEY/OneDrive/Desktop/DS Projects/order_item1(short).csv")
order_items = items.drop(['product_id'], axis=1)
order_items.rename(columns={'order_item_id':'item_id', 'shipping_limit_date':'shipping_date'},inplace=True)

In [85]:
order_items_clean=order_items.drop(['shipping_date'],axis=1)
order_items_clean.head()

,order_id,item_id,seller_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,48436dade18ac8b2bce089ec2a041202,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,5b51032eddd242adc84c38acab88f23d,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,df560393f3a51e74553ab94004ba5c87,199.90,18.14


In [93]:
order_payments = pd.read_csv("C:/Users/GUNGUN PANDEY/OneDrive/Desktop/DS Projects/order_payment(short).csv")
order_payments.rename(columns={'payment_sequential':'payment_seq', 'payment_installments':'installments','product_category_name':'category'},inplace=True)
order_payments.head()

,order_id,payment_seq,payment_type,installments,payment_value,category
0,00010242fe8c5a6d1ba2dd792cb16214,1,credit_card,8,99.33,perfumaria
1,00018f77f2f0320c557190d7a144bdd3,1,credit_card,1,24.39,artes
2,000229ec398224ef6ca0657da4fc703e,1,credit_card,1,65.71,esporte_lazer
3,00024acbcdf0a6daa1e931b038114c75,1,credit_card,8,107.78,bebes
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,credit_card,2,128.45,utilidades_domesticas


In [76]:
product_dataset = pd.read_csv("C:/Users/GUNGUN PANDEY/OneDrive/Desktop/DS Projects/olist_products_dataset.csv")
product_dataset.rename(columns={'product_category_name':'category', 'product_name_lenght':'name_length','product_description_lenght':'desc_length','product_photos_qty':'photos_qty','product_weight_g':'weigth_g','product_length_cm':'length_cm','product_height_cm':'height_cm','product_width_cm':'width_cm'},inplace=True)
product_dataset.head()

,product_id,category,name_length,desc_length,photos_qty,weigth_g,length_cm,height_cm,width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [77]:
cur,conn = create_database()

In [86]:
items_table = ("""CREATE TABLE IF NOT EXISTS items(
order_id VARCHAR PRIMARY KEY,
item_id INT,
seller_id VARCHAR,
price FLOAT,
freight_value FLOAT
)""")

cur.execute(items_table)
conn.commit()

In [79]:
payments_table = ("""CREATE TABLE IF NOT EXISTS payments(
order_id VARCHAR,
payment_seq FLOAT,
payment_type VARCHAR,
installments FLOAT,
payment_value FLOAT,
category VARCHAR
)""")
cur.execute(payments_table)
conn.commit()

In [80]:
product_table = ("""CREATE TABLE IF NOT EXISTS product(
product_id VARCHAR,
category VARCHAR,
name_length FLOAT,
desc_length FLOAT,
photos_qty FLOAT,
weigth_g FLOAT,
length_cm FLOAT,
height_cm FLOAT,
width_cm FLOAT
)""")
cur.execute(product_table)
conn.commit()

In [90]:
items_table_insert = ("""INSERT INTO items(
order_id,
item_id,
seller_id,
price,
freight_value)
VALUES(%s,%s,%s,%s,%s)
""")

In [92]:
for i, row in order_items_clean.iterrows():
    cur.execute(items_table_insert, list(row))

In [95]:
payments_table_insert = ("""INSERT INTO payments(
order_id,
payment_seq,
payment_type,
installments,
payment_value,
category)
VALUES(%s,%s,%s,%s,%s,%s)
""")

In [97]:
for i, row in order_payments.iterrows():
    cur.execute(payments_table_insert, list(row))

In [98]:
product_table_insert = ("""INSERT INTO product(
product_id,
category,
name_length,
desc_length,
photos_qty,
weigth_g,
length_cm,
height_cm,
width_cm)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)
""")

In [99]:
for i, row in product_dataset.iterrows():
    cur.execute(product_table_insert, list(row))

In [100]:
conn.commit()

In [101]:
conn.close()